In [ ]:

import sklearn
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from tensorflow import keras
import xarray as xr
#import kerastuner as kt
from sklearn.linear_model import LinearRegression

tfd = tfp.distributions

## training dw/dt

In [ ]:


def new_sample(df,nn):
    dlen=len(df)
    if nn<dlen:
        return df.sample(n=nn)
    else:
        return df.sample(frac=1)
    
w_bin=[0,1,2,3,4,1000]

var_select=['B','rl','w','t_p','rt_p','thv_env_grad','ent_c_pre','dwdt_pre','ent_t_pre','det_t_pre','ent_c','dwdt','ent_t','det_t']
model_index=1
data_dir="../bomex/data"
dataset_bomex=xr.open_dataset(data_dir+"/"+"sample_clouds_bomex.nc").isel(time=slice(1,119)).to_dataframe().reset_index()
data_dir="../rico/data"
dataset_rico=xr.open_dataset(data_dir+"/"+"sample_clouds_rico.nc").isel(time=slice(1,239)).to_dataframe().reset_index()

dataset=pd.concat([
                   dataset_bomex[(dataset_bomex['zt']>25) & (dataset_bomex['ent_c']>0) & (dataset_bomex['ent_c_pre']>0) & (dataset_bomex['ent_t']>0) & (dataset_bomex['ent_t_pre']>0) & (dataset_bomex['det_t']>0) & (dataset_bomex['det_t_pre']>0) ][var_select].dropna(),
                   dataset_rico[(dataset_rico['zt']>40)   & (dataset_rico['ent_c']>0)  & (dataset_rico['ent_c_pre']>0)  & (dataset_rico['ent_t']>0)  & (dataset_rico['ent_t_pre']>0)  & (dataset_rico['det_t']>0)  & (dataset_rico['det_t_pre']>0)  ][var_select].dropna()
])

dataset['ent_c']=np.log(dataset['ent_c'])
dataset['ent_c_pre']=np.log(dataset['ent_c_pre'])

dataset['ent_t']=np.log(dataset['ent_t'])
dataset['ent_t_pre']=np.log(dataset['ent_t_pre'])
dataset['det_t']=np.log(dataset['det_t'])
dataset['det_t_pre']=np.log(dataset['det_t_pre'])

In [ ]:
out_var=['ent_c','dwdt','ent_t','det_t']

train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)


train_stats = train_dataset.describe(percentiles=[.001,.05, .95,.999])
train_stats['ent_c_pre']=train_stats['ent_c']
train_stats['dwdt_pre']=train_stats['dwdt']
train_stats['ent_t_pre']=train_stats['ent_t']
train_stats['det_t_pre']=train_stats['det_t']
train_stats = train_stats.transpose()

def norm(x,varlist):
    y=x.copy()
    y[varlist]=(y[varlist]) / train_stats['std'][varlist]
    return y

train_dataset = norm(train_dataset,var_select)
test_dataset = norm(test_dataset,var_select)

train_stats.to_csv("saved_model/model_stat_%d.csv" % model_index)


train_labels  = train_dataset[out_var].copy()
train_dataset = train_dataset.drop(out_var, axis=1)
test_labels   = test_dataset[out_var].copy()
test_dataset  = test_dataset.drop(out_var, axis=1)


In [ ]:
import IPython

negloglik = [lambda y1, p_y1:  -p_y1.log_prob(y1), lambda y2, p_y2:  -p_y2.log_prob(y2) , lambda y3, p_y3:  -p_y3.log_prob(y3), lambda y4, p_y4:  -p_y4.log_prob(y4)]

def build_model():
    inputs = tf.keras.Input(shape=[len(train_dataset.keys())])
    x = tf.keras.layers.Dense(units=16, activation='selu')(inputs[...,:6])
    x = tf.keras.layers.Dense(16, activation='selu')(x)
    x = tf.keras.layers.Dense(16, activation='selu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(12)(x)

    
    loc1   = tf.math.softplus(0.1 *x[..., 0:1])*(x[..., 1:2]-inputs[...,6:7])*60.0 + inputs[...,6:7]
    scale1 = tf.math.softplus(0.1 *x[..., 2:3])*tf.sqrt(60.0)
    
    loc2   = tf.math.softplus(0.1 *x[..., 3:4])*(x[..., 4:5]-inputs[...,7:8])*60.0 + inputs[...,7:8]
    scale2 = tf.math.softplus(0.1 *x[..., 5:6])*tf.sqrt(60.0)
    
    loc3   = tf.math.softplus(0.1 *x[..., 6:7])*(x[..., 7:8]-inputs[...,8:9])*60.0 + inputs[...,8:9]
    scale3 = tf.math.softplus(0.1 *x[..., 8:9])*tf.sqrt(60.0)
    
    loc4   = tf.math.softplus(0.1 *x[..., 9:10])*(x[..., 10:11]-inputs[...,9:10])*60.0 + inputs[...,9:10]
    scale4 = tf.math.softplus(0.1 *x[..., 11:12])*tf.sqrt(60.0)
    
    
    scale1=tf.clip_by_value(scale1,clip_value_min=5e-3,clip_value_max=10.0)
    scale2=tf.clip_by_value(scale2,clip_value_min=5e-3,clip_value_max=10.0)
    scale3=tf.clip_by_value(scale3,clip_value_min=5e-3,clip_value_max=10.0)
    scale4=tf.clip_by_value(scale4,clip_value_min=5e-3,clip_value_max=10.0)
    
    x1=tf.concat([loc1,scale1],axis=-1)
    x2=tf.concat([loc2,scale2],axis=-1)
    x3=tf.concat([loc3,scale3],axis=-1)
    x4=tf.concat([loc4,scale4],axis=-1)
    
    outputs1 = tfp.layers.DistributionLambda(
        lambda t: tfd.Normal(loc=t[...,:1] ,scale=t[..., 1:]   ))(x1)
    outputs2 = tfp.layers.DistributionLambda(
        lambda t: tfd.Normal(loc=t[...,:1] ,scale=t[..., 1:]   ))(x2)
    outputs3 = tfp.layers.DistributionLambda(
        lambda t: tfd.Normal(loc=t[...,:1] ,scale=t[..., 1:]   ))(x3)
    
    outputs4 = tfp.layers.DistributionLambda(
        lambda t: tfd.Normal(loc=t[...,:1] ,scale=t[..., 1:]   ))(x4)


    model = tf.keras.Model(inputs=inputs, outputs=[outputs1,outputs2,outputs3,outputs4], name='tiny_model')
    
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss=negloglik)
    return model

    
model = build_model()



In [ ]:


EPOCHS = 1000

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

history = model.fit(
    train_dataset, [train_labels['ent_c'],train_labels['dwdt'],train_labels['ent_t'],train_labels['det_t']],
    epochs=EPOCHS, callbacks=[early_stop], validation_split = 0.2)


In [ ]:
#tmp=plt.hist(np.array(test_predictions[:,2]),bins=100)
#tmp=plt.hist2d(np.array(test_predictions[:,1]),np.array(test_predictions[:,2]),bins=40)


model.save("saved_model/model_%d.h5" % model_index)